In [10]:
# CONNECT
from qdrant_client import QdrantClient,models
client=QdrantClient(url="http://localhost:6333")

In [11]:
'''
ubuntu
docker run -p 6333:6333 -p 6334:6334 \
    -v $(pwd)/qdrant_storage:/qdrant/storage:z \
    qdrant/qdrant
'''

'\nubuntu\ndocker run -p 6333:6333 -p 6334:6334     -v $(pwd)/qdrant_storage:/qdrant/storage:z     qdrant/qdrant\n'

In [12]:
'''
window:
docker run -p 6333:6333 -v /absolute/path/here/qdrant_storage:/qdrant/storage qdrant/qdrant
'''

'\nwindow:\ndocker run -p 6333:6333 -v /absolute/path/here/qdrant_storage:/qdrant/storage qdrant/qdrant\n'

In [13]:
my_collection="Vector_database_summarization"

In [14]:
#TẠO COLLECTION

client.delete_collection(collection_name=my_collection)
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE),
)



True

In [16]:
import torch
from collecting_data_sql import data_sql
dataset=data_sql()
from model_embedding import embedding_text

# checkpoint_model = 'dangvantuan/vietnamese-embedding'  # 512 -> 768
checkpoint_model = "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"  # 256 -> 768

client.upload_points(
    collection_name=my_collection,
    points=[
        models.PointStruct(
            id=idx,
            vector=embedding_text(doc['NoiDungTomTat'], checkpoint_model),
            payload=doc,
        )for idx,doc in enumerate(dataset)
    ],
)

# 66m 58.3s


Token sau khi tokenizer: ['Cách@@', 'thức', 'thự@@', 'ch@@', 'iện', 'Nộp', 'trự@@', 'c@@', 'tiếp', 'đường', 'bư@@', 'u@@', 'điện', 'trự@@', 'c@@', 'tuyến', 'hồ@@', 'sơ', 'điệ@@', 'n@@', 'tử', 'Bộ', 'V@@', 'ă@@', 'nh@@', 'ó@@', 'a_@@', 'Thể@@', 'thao', 'Dul@@', 'ịch', 'Vụ', 'Tổ@@', 'chức', 'cá@@', 'n@@', 'bộ', '3@@', 'Đối', 'Ban', 'đảng', 'thự@@', 'ch@@', 'iện', 'thể@@', 'thức', 'vă@@', 'n@@', 'bản', 'Đảng', 'M@@', 'ẫ@@', 'u@@', 'số', '02', 'Mẫu', 'báo@@', 'cáo', 'á@@', 'p@@', 'dụng', 'đố@@', 'i@@', 'với', 'tập@@', 'thể', 'B@@', 'Á@@', 'O@@', 'CÁ@@', 'O', 'THÀ@@', 'NH@@', 'T@@', 'ÍCH', 'Đ@@', 'Ề@@', 'NGH@@', 'Ị', 'KH@@', 'EN', '12@@', 'Ký', 'hiệu', 'so@@', 'ạ@@', 'n@@', 'thả@@', 'o@@', 'văn', 'bản', 'số@@', 'lượng', 'bản', 'phá@@', 'thành', 'Vụ', 'Tổ@@', 'chức', 'cá@@', 'n@@', 'bộ', 'th@@', 'ẩ@@', 'm@@', 'định', 'hồ@@', 'sơ', 'kh@@', 'en@@', 'thưởng', 'trình', 'cấp', 'th@@', 'ẩ@@', 'm@@', 'quyền', 'thờ@@', 'i@@', 'hạn', '25', 'là@@', 'm@@', 'việc', 'kể', 'đủ', 'hồ@@', 'sơ', 'quy@@', 'đị

SEARCH

In [32]:
# search toàn bộ văn bản
def search_full(query_vector, limit,my_collection):
   my_collection=my_collection
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      limit=limit
   )
   for i in search_result:
      print(i)


# chỉ in ra 1 số thông tin cần thiết

def search_with_some_inf(query_vector, limit, my_collection):
   
   my_collection=my_collection
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      # query_filter=models.Filter(
      #    must=[models.FieldCondition(key="dieu", match=models.MatchValue(value="2"))]
      # ),
      limit=limit,
      search_params=models.SearchParams(
               exact=True,  # Turns on the exact search mode KNN
            ),
   )

   #print question
   if len(search_result)==0:
      print("Không có thông tin tìm kiếm")
   else:
      print("Kết quả thông tin tìm kiếm là:")
      answer_array=[]
      for i in search_result:

         answer_json={
            'index':i.id,
            'score':i.score,
            "NameFile": i.payload['NamFile'],
            "NoiDungTomTat": i.payload['NoiDungTomTat'], ################## noi dung

            # "id-van-ban": i.payload['id-van-ban'],
            # "SoHieu": i.payload['SoHieu'],
            # "LoaiVanBan": i.payload['LoaiVanBan'],
            # "NoiBanHanh": i.payload['NoiBanHanh'],
            # "NguoiKy": i.payload['NguoiKy'],
            # "NgayBanHanh": i.payload['NgayBanHanh'],
            # "NgayHieuLuc": i.payload['NgayHieuLuc'],
            # "NgayCongBao": i.payload['NgayCongBao'],
            # "SoCongBao": i.payload['SoCongBao'],
            # "TinhTrang": i.payload['TinhTrang'],
            
            # "LinhVuc": i.payload['LinhVuc'],
            
         }
         if len(answer_array)==0:
            answer_array.append(answer_json)
         else:
            count=0
            for answer in answer_array:
            #id là của 1 văn bản , phải thêm chương, điều, mục
               if (answer_json['NameFile']==answer['NameFile']):
                  count=count+1
            if count==0:
               answer_array.append(answer_json)

      for i in answer_array:
         print(i)



In [34]:
import torch
from model_embedding import embedding_text
text=' CP ngày 05122011 về công_chức xã phường thị_trấn Nghị_định số 342019NĐ CP ngày 2442019 của Chính_phủ sửa_đổi bổ_sung một_số quy_định về cán_bộ công_chức cấp xã và người hoạt_động không chuyên_trách ở cấp xã ở thôn tổ dân_phố đã hết hiệu_lực 0182023'

search_with_some_inf(embedding_text(text, checkpoint_model),15,my_collection)

#438_QĐ_UBND_m_604873.doc
# 438_QĐ_UBND_m_604873.doc
#'06_2024_QĐ_UBND_m_605513.doc
# 689_QĐ_UBND_m_605655.doc
# 295_QĐ_UBND_m_604452.doc
# 527_QĐ_UBND_m_604654.doc
# 375_QĐ_UBND_m_604476.doc

Token sau khi tokenizer: ['CP', '05@@', '12@@', '2011', 'cô@@', 'ng@@', 'chức', 'xã', 'phường', 'thị@@', 'trấn', 'Nghị@@', 'định', '34@@', '201@@', '9@@', 'NĐ', 'CP', '24@@', '4@@', '2019', 'Chí@@', 'nh@@', 'phủ', 'sử@@', 'a@@', 'đổi', 'bổ@@', 'sung', 'mộ@@', 't@@', 'số', 'quy@@', 'định', 'cá@@', 'n@@', 'bộ', 'cô@@', 'ng@@', 'chức', 'cấp', 'xã', 'hoạ@@', 't@@', 'động', 'chuy@@', 'ê@@', 'n@@', 'trách', 'cấp', 'xã', 'thôn', 'tổ', 'dâ@@', 'n@@', 'phố', 'hiệ@@', 'u@@', 'lực', '0@@', '18@@', '2023'] ==
Chiều dài token: 62 ===
Kết quả thông tin tìm kiếm là:
{'index': 987, 'score': 1.0, 'NameFile': '438_QĐ_UBND_m_604873.doc', 'NoiDungTomTat': ' CP ngày 05122011 về công_chức xã phường thị_trấn Nghị_định số 342019NĐ CP ngày 2442019 của Chính_phủ sửa_đổi bổ_sung một_số quy_định về cán_bộ công_chức cấp xã và người hoạt_động không chuyên_trách ở cấp xã ở thôn tổ dân_phố đã hết hiệu_lực 0182023'}
{'index': 275, 'score': 0.73230994, 'NameFile': '06_2024_QĐ_UBND_m_605513.doc', 'NoiDungTomTat': 'Bãi_b

In [24]:
import torch
from model_embedding import embedding_text
text=' Thủ tục hành chính cấp Trung ương trong lĩnh vực Thi đua khen thưởng có số thứ tự 10 11 điểm A4 mục A phần I ban hành kèm theo Quyết định số 787/QĐ BVHTTDL ngày 31 tháng 3 năm 2023 của Bộ trưởng Bộ Văn hóa Thể thao và Du lịch về việc công bố thủ tục hành chính nội bộ trong hệ thống hành chính nhà nước thuộc phạm vi chức năng quản lý của Bộ Văn hóa Thể thao và Du lịch hết hiệu lực thi hành kể từ ngày Quyết định này có hiệu lực thi hành. 5 Các hình thức khen thưởng công trạng đột xuất chuyên đề cống hiến niên hạn đối ngoại 6 Các danh hiệu thi đua thuộc thẩm quyền của Thủ tướng Chính phủ Chính phủ.  Mẫu số 09 (Áp dụng đối với tổ chức)  BÁO CÁO THÀNH TÍCH ĐỀ NGHỊ TẶNG. 4 Báo cáo thành tích liên tục từ 25 năm trở lên đến thời điểm đề nghị đối với Huân chương Sao vàng liên tục từ 10 năm trở lên đến thời điểm đề nghị đối với Huân chương Hồ Chí Minh Huân chương Độc lập Huân chương Quân công liên tục từ 05 năm trở lên đến thời điểm đề nghị đối với Huân chương Bảo vệ Tổ quốc Huân chương Lao động liên tục từ 05 năm trở lên đến thời điểm đề nghị đối với Bằng khen của Thủ tướng Chính phủ 01 năm đối với Cờ thi đua của Chính phủ Cờ thi đua của bộ ban ngành tỉnh và Giấy khen 01 năm đối với Tập thể lao động xuất sắc Tập thể lao động tiên tiến 02 năm liên tục đối với Bằng khen của bộ ban ngành tỉnh. Đối với tập thể nhỏ không thuộc đối tượng tặng Cờ thi đua của bộ ban ngành tỉnh thì ghi rõ nội dung này trong báo cáo thành tích'

search_with_some_inf(embedding_text(text, checkpoint_model),3,my_collection)
# '988_QĐ_BVHTTDL_m_605999.doc'

Token sau khi tokenizer: ['Thủ_tục', 'hành_chính', 'cấp', 'Trung_ương', 'lĩnh_vực', 'Thi_đua', 'khen_thưởng', 'thứ', 'tự', '10', '11', 'điểm', 'A4', 'mục', 'A', 'phần', 'I', 'ban_hành', 'kèm', 'Quyết_định', '787', 'QĐ', 'B@@', 'VHTTDL', '31', '3', '2023', 'Bộ_trưởng', 'Bộ', 'Văn_@@', 'h@@', 'ó@@', 'a', 'Thể_thao', 'Du_lịch', 'công_bố', 'thủ_tục', 'hành_chính', 'nội_bộ', 'hệ_thống', 'hành_chính', 'nhà_nước', 'thuộc', 'phạm_vi', 'chức_năng', 'quản_lý', 'Bộ', 'Văn_@@', 'h@@', 'ó@@', 'a', 'Thể_thao', 'Du_lịch', 'hiệu_lực', 'thi_hành', 'kể', 'Quyết_định', 'hiệu_lực', 'thi_hành', '5', 'Các', 'hình_thức', 'khen_thưởng', 'công_trạng', 'đột_xuất', 'chuyên_đề', 'cống_hiến', 'niên_hạn', 'đối_ngoại', '6', 'Các', 'danh_hiệu', 'thi_đua', 'thuộc', 'thẩm_quyền', 'Thủ_tướng', 'Chính_phủ', 'Chính_phủ', 'Mẫu_@@', 'số', '09', '(', 'Áp_dụng', 'đối_với', 'tổ', 'B@@', 'Á@@', 'O@@', '_C@@', 'ÁO', 'THÀ@@', 'NH@@', '_T@@', 'ÍCH', 'Đ@@', 'Ề@@', '_@@', 'NGH@@', 'Ị', 'T@@', 'Ặ@@', 'NG', '4', 'Báo_cáo', 'thành_tích

In [24]:
import torch
from SimeCSE_Vn_model import embedding_text
text = ' Công bố kèm theo Quyết định này thủ tục hành chính nội bộ mới ban hành và sửa đổi  bổ sung trong lĩnh vực Thi đua  khen thưởng giữa các cơ quan  đơn vị trực thuộc Bộ Văn hóa  Thể thao và Du lịchh'

my_collection='SimeCSE_Vn_doan'
answer_array=search_with_some_inf(embedding_text(text),10,my_collection)


torch.Size([1, 36])
['Công_bố', 'kèm', 'theo', 'Quyết_định', 'này', 'thủ_tục', 'hành_chính', 'nội_bộ', 'mới', 'ban_hành', 'và', 'sửa_đổi', 'bổ_sung', 'trong', 'lĩnh_vực', 'Thi_đua', 'khen_thưởng', 'giữa', 'các', 'cơ_quan', 'đơn_vị', 'trực_thuộc', 'Bộ', 'Văn_@@', 'h@@', 'ó@@', 'a', 'Thể_thao', 'và', 'D@@', 'u_l@@', 'ị@@', 'ch@@', 'h']
Kết quả thông tin tìm kiếm là:
{'index': 0, 'score': 0.97110337, 'noi dung': ' Công bố kèm theo Quyết định này thủ tục hành chính nội bộ mới ban hành và sửa đổi  bổ sung trong lĩnh vực Thi đua  khen thưởng giữa các cơ quan  đơn vị trực thuộc Bộ Văn hóa  Thể thao và Du lịch', 'id_van_ban': 1121, 'so-hieu': '987/QĐ-BVHTTDL', 'chuong': 0, 'muc': 0, 'dieu': 1}
{'index': 449, 'score': 0.92570883, 'noi dung': ' Công bố kèm theo Quyết định này thủ tục hành chính nội bộ giữa các cơ quan hành chính nhà nước mới ban hành và sửa đổi  bổ sung trong lĩnh vực Thi đua  khen thưởng thuộc phạm vi chức năng quản lý của Bộ Văn hóa  Thể thao và Du lịch', 'id_van_ban': 1122, '

In [25]:
import torch
from SimeCSE_Vn_model import embedding_text
text = '  Vụ Tổ chức cán bộ thẩm định hồ sơ khen thưởng và trình Hội đồng Thi đua  khen thưởng Bộ trong thời hạn 25 ngày làm việc kể từ ngày nhận đủ hồ sơ theo quy định (trừ khen thưởng đột xuất) \
Đối với trường hợp hồ sơ không đầy đủ  trong thời hạn 05 ngày làm việc  Vụ Tổ chức cán bộ thông báo tới cơ quan  đơn vị trình hoàn thiện hồ sơ'

my_collection='SimeCSE_Vn_doan'
answer_array=search_with_some_inf(embedding_text(text),20,my_collection)


torch.Size([1, 52])
['Vụ', 'Tổ_chức', 'cán_bộ', 'thẩm_định', 'hồ_sơ', 'khen_thưởng', 'và', 'trình', 'Hội_đồng', 'Thi_đua', 'khen_thưởng', 'Bộ', 'trong', 'thời_hạn', '25', 'ngày', 'làm_việc', 'kể', 'từ', 'ngày', 'nhận', 'đủ', 'hồ_sơ', 'theo', 'quy_định', '(', 'trừ', 'khen_thưởng', 'đột_xuất', ')', 'Đối_với', 'trường_hợp', 'hồ_sơ', 'không', 'đầy_đủ', 'trong', 'thời_hạn', '05', 'ngày', 'làm_việc', 'Vụ', 'Tổ_chức', 'cán_bộ', 'thông_báo', 'tới', 'cơ_quan', 'đơn_vị', 'trình', 'hoàn_thiện', 'hồ_sơ']
Kết quả thông tin tìm kiếm là:
{'index': 10, 'score': 1.0, 'noi dung': '  Vụ Tổ chức cán bộ thẩm định hồ sơ khen thưởng và trình Hội đồng Thi đua  khen thưởng Bộ trong thời hạn 25 ngày làm việc kể từ ngày nhận đủ hồ sơ theo quy định (trừ khen thưởng đột xuất) Đối với trường hợp hồ sơ không đầy đủ  trong thời hạn 05 ngày làm việc  Vụ Tổ chức cán bộ thông báo tới cơ quan  đơn vị trình hoàn thiện hồ sơ', 'id_van_ban': 1121, 'so-hieu': '987/QĐ-BVHTTDL', 'chuong': 0, 'muc': 0, 'dieu': 3}
{'index': 97, 

In [13]:
import torch
from SimeCSE_Vn_model import embedding_text
text = ' Các trường hợp đề nghị khen thưởng trên đã được xét duyệt  thẩm định đảm bảo thế nào'

my_collection='SimeCSE_Vn_doan'
answer_array=search_with_some_inf(embedding_text(text),10,my_collection)


torch.Size([1, 14])
['Các', 'trường_@@', 'hợp_@@', 'đề_nghị', 'khen_thưởng', 'trên', 'đã', 'được', 'xét_duyệt', 'thẩm_định', 'đảm_bảo', 'thế_nào']
Kết quả thông tin tìm kiếm là:
{'index': 491, 'score': 0.89580476, 'noi dung': 'Các trường hợp đề nghị khen thưởng trên đã được 1 xét duyệt  thẩm định đảm bảo đúng quy định về đối tượng  tiêu chuẩn  hồ sơ  thủ tục theo quy định hiện hành', 'id_van_ban': 1122, 'so-hieu': '988/QĐ-BVHTTDL', 'chuong': 0, 'muc': 0, 'dieu': 3}
{'index': 587, 'score': 0.89580476, 'noi dung': 'Các trường hợp đề nghị khen thưởng trên đã được 1 xét duyệt  thẩm định đảm bảo đúng quy định về đối tượng  tiêu chuẩn  hồ sơ  thủ tục theo quy định hiện hành', 'id_van_ban': 1122, 'so-hieu': '988/QĐ-BVHTTDL', 'chuong': 0, 'muc': 0, 'dieu': 3}
{'index': 649, 'score': 0.89580476, 'noi dung': 'Các trường hợp đề nghị khen thưởng trên đã được 1 xét duyệt  thẩm định đảm bảo đúng quy định về đối tượng  tiêu chuẩn  hồ sơ  thủ tục theo quy định hiện hành', 'id_van_ban': 1122, 'so-hieu'